## Egyszerű  Chatbot / 2


#TF-IDF, Koszinusz hasonlóság, Szótövezés



#Excel betöltése, beolvasás Pandas-al

In [28]:
# excel letöltése a Githubról
!wget https://raw.githubusercontent.com/minorharpman/ai_prog_pub/main/hirlevel_10/fogaszat_szoveg.xlsx

--2026-02-27 15:21:27--  https://raw.githubusercontent.com/minorharpman/ai_prog_pub/main/hirlevel_10/fogaszat_szoveg.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10174 (9.9K) [application/octet-stream]
Saving to: ‘fogaszat_szoveg.xlsx.1’

fogaszat_szoveg.xls 100%[===================>]   9.94K  --.-KB/s    in 0s      

2026-02-27 15:21:27 (68.7 MB/s) - ‘fogaszat_szoveg.xlsx.1’ saved [10174/10174]



In [29]:
import pandas as pd

# Excel beolvasása
df = pd.read_excel("fogaszat_szoveg.xlsx")
df.columns.tolist()



['Cím', 'Szöveg']

In [30]:
# Új oszlop létrehozása (Cím + Szöveg)
df["adat"] = df["Cím"].fillna("") + " " + df["Szöveg"].fillna("")
df["adat"]

,adat
0,FOGFEHÉRÍTÉS A fogfehérítés esztétikai kezelés...
1,"FOGHÚZÁS A foghúzás akkor szükséges, ha a fog ..."
2,FOGKŐ ELTÁVOLÍTÁS A fogkő eltávolítás segít me...
3,FOGPÓTLÁS A fogpótlás hiányzó fogak esetén meg...
4,FOGSZABÁLYOZÁS A fogszabályozás célja a fogak ...
5,GYÖKÉRKEZELÉS A gyökérkezelés a fog belsejében...
6,GYERMEKFOGÁSZAT A gyermekfogászat a gyerekek f...
7,RÖNTGEN A fogászati röntgen segít a pontos dia...
8,TÖMÉSEK A fogtömés szuvas fogak helyreállításá...
9,ÁLTALÁNOS INFORMÁCIÓK Rendelőnk hétfőtől pénte...


#Szótövezés rész kezdete,  
* csak szótőre lehet keresni, rokonértelmű szavakra nem! (következő rész)

In [31]:
# fájlok letöltése a Githubról

!wget https://raw.githubusercontent.com/minorharpman/ai_prog_pub/main/nlp/hu_HU.aff
!wget https://raw.githubusercontent.com/minorharpman/ai_prog_pub/main/nlp/hu_HU.dic

!apt-get update
!apt-get install -y libhunspell-dev
!pip install hunspell

#https://github.com/LibreOffice/dictionaries/tree/master/hu_HU
import hunspell

h = hunspell.HunSpell(
    "hu_HU.dic",
    "hu_HU.aff"
)

import hunspell

# Hunspell inicializálás
import hunspell

h = hunspell.HunSpell(
    "hu_HU.dic",
    "hu_HU.aff"
)

def lemmatize_word(word):
    stems = h.stem(word)
    if stems:
        return stems[0].decode("utf8")
    return word

def lemmatize_text(text):
    words = text.split()
    return " ".join(lemmatize_word(w) for w in words)



--2026-02-27 15:21:38--  https://raw.githubusercontent.com/minorharpman/ai_prog_pub/main/nlp/hu_HU.aff
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2236063 (2.1M) [application/octet-stream]
Saving to: ‘hu_HU.aff.1’

hu_HU.aff.1         100%[===================>]   2.13M  --.-KB/s    in 0.06s   

2026-02-27 15:21:38 (35.9 MB/s) - ‘hu_HU.aff.1’ saved [2236063/2236063]

--2026-02-27 15:21:38--  https://raw.githubusercontent.com/minorharpman/ai_prog_pub/main/nlp/hu_HU.dic
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1756889 (1.7

# Szótövezés rész vége

In [33]:
# szótövezés az 'adat' oszlopon
df["adat"] = df["adat"].apply(lemmatize_text)
df["adat"]

,adat
0,"fogfehérítés a fogfehérítés esztétika kezelés,..."
1,"foghúzás a foghúzás akkor szükséges, ha a fog ..."
2,fogkő eltávolítás a fogkő eltávolítás segít me...
3,fogpótlás a fogpótlás hiányzó fog eset megoldá...
4,fogszabályozás a fogszabályozás cél a fog hely...
5,gyökérkezelés a gyökérkezelés a fog belseje lé...
6,gyermekfogászat a gyermekfogászat a gyerek fog...
7,röntgen a fogászati röntgen segít a pontos dia...
8,tömés a fogtömés szuvas fog helyreállítás szol...
9,általános információ rendelő hétfő péntek 8:00...


In [34]:
# Könyvtárak importálása
# Szövegfeldolgozáshoz és hasonlóság számításhoz szükséges csomagok
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(
    lowercase=True,
    # Ne csak egyes szavakat tanulj meg, hanem két egymás melletti szóból álló kifejezéseket is.
    # ngram_range=(1,2)
    # csak egyetlen szavakat tanulj meg
    ngram_range = (1, 1)
)

tfidf_matrix = vectorizer.fit_transform(df["adat"])


In [35]:
def chatbot_valasz(kerdes, kuszob=0.1):
    # Kérdés vektorozása
    kerdes_vektor = vectorizer.transform([kerdes])

    # Koszinusz hasonlóság
    hasonlosagok = cosine_similarity(kerdes_vektor, tfidf_matrix)

    # Legjobb találat
    index = hasonlosagok.argmax()
    ertek = hasonlosagok[0][index]
    print(f"Hasonlóság: {ertek}")
    print(f"Index: {index}")

    # Ha nincs elég jó egyezés
    if ertek < kuszob:
        return (
            "Sajnos erre nem találtam megfelelő információt.\n\n"
            "📞 Telefon: +36 1 234 5678\n"
            "✉️ Email: info@demofogaszat.hu\n"
            "⏰ Nyitvatartás: H–P 8:00–18:00"
        )

    # Megfelelő témakör
    cim = df.iloc[index]["Cím"]
    szoveg = df.iloc[index]["Szöveg"]

    return f"**{cim}**\n\n{szoveg}"


In [38]:
ugyfel_szoveg = input("Én:\n")
print("___________")

print("\nChatbot asszisztens:\n"+chatbot_valasz(ugyfel_szoveg))

Én:
tömés
___________
Hasonlóság: 0.46465237353429156
Index: 8

Chatbot asszisztens:
**TÖMÉSEK**

A fogtömés szuvas fogak helyreállítására szolgál.
Esztétikus, fogszínű tömések érhetők el.
A kezelés megakadályozza a további romlást.
Gyors és hatékony megoldás.
Helyi érzéstelenítésben történik.


In [ ]:

# 0.1 küszöbnél, a tartalomban levő szövegre is talál
print(chatbot_valasz("Fájdalmas a foghúzás?"))


Hasonlóság: 0.530228128548479
Index: 1
**FOGHÚZÁS**

A foghúzás akkor szükséges, ha a fog már nem menthető.
Gyulladás vagy súlyos szuvasodás esetén alkalmazzuk.
A beavatkozás helyi érzéstelenítésben történik.
A foghúzás után fontos az utókezelési tanácsok betartása.
A gyógyulási idő általában néhány nap.


In [ ]:
#print(chatbot_valasz("iskolás"))
print(chatbot_valasz("gyerek"))

Sajnos erre nem találtam megfelelő információt.

📞 Telefon: +36 1 234 5678
✉️ Email: info@demofogaszat.hu
⏰ Nyitvatartás: H–P 8:00–18:00


#Hogyan dönti el a chatbot, mi a „fontos szó”?  
(TF-IDF alapú magyarázat)

---

##  A chatbot **NEM tudja**, mi a fontos szó

A TF-IDF **nem érti** a szavak jelentését.  
Nem tudja, hogy a „gyermek” vagy a „fogászati” fontos-e.

Amit csinál:

1. **Fogja a kérdést**
2. **Tokenizálja** (szavakra bontja)
3. Minden szónak ad egy **TF-IDF súlyt**
4. A súlyokból készít egy **vektort**
5. A kérdés vektorát összehasonlítja a dokumentumok vektoraival  
   → **koszinusz hasonlóság alapján**

---

##  A TF-IDF dönti el, mely szó „fontos”

A kérdés vektorában minden szó kap egy saját súlyt:

Hogy mely szó lesz „fontos”, azt két dolog határozza meg:

###  TF – Term Frequency  
Hányszor szerepel a szó a kérdésben?

###  IDF – Inverse Document Frequency  
Mennyire ritka a szó a teljes tudásbázisban?

---

###  Következmény

- **Minél ritkább egy szó → annál nagyobb a súlya**  
- **Minél gyakoribb → annál kisebb a súlya**

Ez alapján a vektor „irányát” főként a ritkább szavak befolyásolják.




In [ ]:
def fontos_szavak_a_kerdesben(kerdes):
    kerdes_vec = vectorizer.transform([kerdes])
    tokenek = vectorizer.get_feature_names_out()
    scores = kerdes_vec.toarray()[0]

    # Nem nullák → jelen vannak a kérdésben
    indexek = scores.argsort()[::-1]  # csökkenő sorrend

    print("Kérdés fontos TF-IDF szavai:")
    for i in indexek:
        if scores[i] > 0:
            print(f"{tokenek[i]:20s} → {scores[i]:.4f}")

fontos_szavak_a_kerdesben("A fogfehérítés milyen környezetben történik?  ")

Kérdés fontos TF-IDF szavai:
fogfehérítés         → 0.8550
történik             → 0.5186


##  TF-IDF mátrix
* függőlegesen a dokumentumok vannak pl.: FOGFEHÉRÍTÉS, FOGHÚZÁS
* vízszintesen egyszavas tokenek

In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Tokenek (oszlopnevek)
tokenek = vectorizer.get_feature_names_out()


print("Tokenek száma:", len(tokenek))

# TF-IDF mátrix DataFrame-ként
df_tfidf = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tokenek
)

df_tfidf


Tokenek száma: 170


,00,18,36,akkor,akár,alacsony,alap,alapvető,alkalmaz,alkalmazható,alkalom,amely,az,barátságos,beavatkozás,belseje,betartás,biztonságos,bla,budapest,cél,címünk,demofogaszat,diagnózis,digitális,dohányzás,egyéni,egészséges,el,elhalt,eljárás,ellátás,elszíneződéseket,eltávolít,eltávolítás,előtt,email,emailben,eredmény,eset,eszköz,esztétika,esztétikus,felnőtt,feltárás,fertőzés,fog,fogfehérítés,fogfelszín,foghúzás,fogkő,foglalkozik,fogpótlás,fogszabályozás,fogszín,fogtömés,fogászati,fontos,fájdalommentes,gyerek,gyermek,gyermekfogászat,gyors,gyulladás,gyógyulási,gyökérkezelés,ha,harapás,hatás,hatékony,helyi,helyreállít,helyreállítás,helyzet,hiányzó,hu,hétfő,hónap,ideg,idő,időpontfoglalás,időtartam,igényel,info,információ,is,javasol,javít,jár,kap,kellemetlenség,kezelés,kiemelt,kivehető,kontroll,korrigálás,kávé,környezet,között,lehetséges,lévő,megakadályoz,megelőz,megelőzés,megoldás,megszüntetés,megtartás,megőriz,menthető,minimális,mosoly,már,nagy,nap,nem,nevelés,nyit,néhány,okoz,pontos,probléma,péntek,rejtett,rendelő,rendszeres,romlás,rágófunkció,rögzített,röntgen,röntgentechnológia,segít,sok,során,sugárterhelésű,szerep,szolgál,szuvas,szuvasodás,szükséges,súlyos,tanács,tart,tartós,tea,tejfog,telefon,természetes,tervezés,tesz,teszt,tiszta,további,tömés,történik,ultrahangos,utca,után,utókezelés,van,világos,vizsgálat,állapotfelmérés,általában,általános,árnyalat,érhető,érzéstelenítés,és,évente,ínybetegség
0,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.318125,0.175371,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.159062,0.135961,0.000000,0.159062,0.000000,0.000000,0.159062,0.000000,0.000000,0.119570,0.000000,0.000000,0.000000,0.000000,0.080077,0.477187,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.135961,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.119570,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240232,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.000000,0.000000,0.000000,0.135961,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.119570,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.000000,0.000000,0.000000,0.159062,0.000000,0.000000,0.000000,0.000000,0.096468,0.000000,0.000000,0.000000,0.000000,0.175371,0.318125,0.000000,0.159062,0.000000,0.000000,0.159062,0.000000,0.000000,0.087685,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.176743,0.000000,0.000000,0.000000,0.000000,0.151073,0.000000,0.000000,0.000000,0.097432,0.000000,0.151073,0.000000,0.176743,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.151073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.088978,0.000000,0.000000,0.530228,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.151073,0.000000,0.000000,0.000000,0.000000,0.000000,0.176743,0.176743,0.000000,0.176743,0.000000,0.000000,0.000000,0.132860,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176743,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176743,0.000000,0.000000,0.176743,0.000000,0.176743,0.176743,0.000000,0.0000